## Imports

In [1]:
from pathlib import Path

import pandas as pd

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

from mlops.processors.churn import ChurnProcessor

from mlops.config import MODEL_CONFIG

## Data

In [2]:
data_path = Path("../data") / "churn" / "data.parquet"
dataset = pd.read_parquet(data_path)

In [3]:
X = dataset.drop(columns=[MODEL_CONFIG["models"]["churn"]["target"]])
y = dataset[MODEL_CONFIG["models"]["churn"]["target"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=MODEL_CONFIG["models"]["churn"]["test_size"]
)

## Training and testing

In [4]:
model = ChurnProcessor().train(X_train, y_train)

/Users/migduroli/dev/loyola/mlops/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/migduroli/dev/loyola/mlops/.venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
model.pipeline

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('outlier_clipper',
                                                                                          OutlierClipper()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         [3, 6,
                                                                          7, 8,
                                                                          9, 10,
                                                                          11,
                                                                          12]),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constan...
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         [4,
                                                                          5])])),
                                       ('mlp_classifier',
                                        MLPClassifier(random_state=123456))]),
             n_jobs=-1,
             param_grid={'mlp_classifier__activation': ['relu', 'tanh'],
                         'mlp_classifier__hidden_layer_sizes': [[8, 1],
                                                                [8, 6, 1],
                                                                [16, 1],
                                                                [16, 8, 1],
                                                                [32, 16, 1]],
                         'mlp_classifier__learning_rate_init': [0.001, 0.01],
                         'mlp_classifier__max_iter': [1000]},
             refit='roc_auc', scoring=['accuracy', 'f1', 'roc_auc'])

In [6]:
predictions, probabilities = model.predict(X_test)

In [7]:
metrics = model.compute_metrics(X_test, y_test)

In [8]:
metrics

{'accuracy': np.float64(0.8625923078890246),
 'roc_auc_score': np.float64(0.7244685668039095),
 'f1_score': 0.597667638483965}

In [19]:
model_path = Path("..") / "artifacts" / "models" / "churn" / "model.flm"

## Dumping and loading

In [20]:
model.dump(metrics, model_path=model_path)

In [21]:
model_loaded = ChurnProcessor.load(model_path)
model_loaded.compute_metrics(X_test, y_test)

{'accuracy': np.float64(0.8625923078890246),
 'roc_auc_score': np.float64(0.7244685668039095),
 'f1_score': 0.597667638483965}

In [22]:
model_loaded.pipeline

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('outlier_clipper',
                                                                                          OutlierClipper()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         [3, 6,
                                                                          7, 8,
                                                                          9, 10,
                                                                          11,
                                                                          12]),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constan...
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         [4,
                                                                          5])])),
                                       ('mlp_classifier',
                                        MLPClassifier(random_state=123456))]),
             n_jobs=-1,
             param_grid={'mlp_classifier__activation': ['relu', 'tanh'],
                         'mlp_classifier__hidden_layer_sizes': [[8, 1],
                                                                [8, 6, 1],
                                                                [16, 1],
                                                                [16, 8, 1],
                                                                [32, 16, 1]],
                         'mlp_classifier__learning_rate_init': [0.001, 0.01],
                         'mlp_classifier__max_iter': [1000]},
             refit='roc_auc', scoring=['accuracy', 'f1', 'roc_auc'])